---
title: Encoding Music Sample Homework Project
author: Richard Freedman
date: 'June 25, 2024'
format:
  html:
    code-fold: true
    embed-resources: true
plotly-connected: true
jupyter: python3
---


# A Sample Homework Project

This sample homework project will show you in a nutshell what your work for Encoding Music should look like. It follows the [Notebook Style Guide](https://github.com/RichardFreedman/music_style_guide/blob/main/M255_Notebook_Style_Guide.md) and [Music Style Guide](https://github.com/RichardFreedman/music_style_guide).  And it also demonstrates the key sections of every notebook submitted for a class assignment or project:

* The preliminaries:  Quarto YAML cell and code import statement
* The three main sections:  Motivation/Implementation/Interpretation
* The bibliography of sources consulted

### 0. Quarto YAML cell and Import Statement

The Quarto YAML cell will contain the title, name of author(s) and date for your assignment.

The Import Statement is just that:  the code cell where you import all relevant libraries for your work,

### 1. Motivation

What is your point of inquiry?  What are you trying discover?  What do you _expect_ to find? How do these questions relate to themes and problems considered in the course so far?  

What specific readings or musical repertories are involved, and why do they matter in this context (be sure to cite them, and to follow the Music Style Guide as you do so)?

### 2. Implementation

How did you put your query into practice? What did you need to do with or to your data along the way? 

* What _logical_ steps did you follow as you thought about getting there (expressed in pseudo code; perhaps in bullet-point form in your markdown cell)?
* Which tools (or methods) did you select, and why were they right for the job?
* If you chose to create some kind of visualization (a chart, network or map), how did you decide on the right representation for the task at hand?
* What challenges did you face as you worked to put the steps into action in code?  Did you need to clean or tidy your data?  Filter, sort, or group it in some way?
* What were the most important tools or approaches you deployed that others might like to know about?

Remember to use `verbatim code` or **block code** to show your work as needed.
Remember to add **comments** to your code to show the most important steps.

If you learned from or adapted some code suggested in Encoding Music, or another resource, it would be good to cite that documentation, too.

### 3. Interpretation

What did you learn from the results? How were your expectations confirmed (or not)?

What limitations did the data or your methods impose on your capacity to answer the questions you originally posed? 

How do your results change your views of the readings or ideas that motivated your work it the first place?  How do your results shape your listening experience of the music you considered? What next steps do you imagine for future work?

### 4. Bibliography

A list of the sources consulted and cited during your work, including class readings or code resources, Spotify lists, etc.  See the [Notebook Style Guide](https://github.com/RichardFreedman/music_style_guide/blob/main/M255_Notebook_Style_Guide.md) and [Music Style Guide](https://github.com/RichardFreedman/music_style_guide) for details about how to cite books, articles, internet resources, etc.

# 0. Import Libraries

Here I import the essential libraries for this assignment.


In [ ]:
# import libraries
from community import community_louvain
from copy import deepcopy
from IPython.display import SVG
import altair as alt
import glob as glob
import numpy as np
import os
import pandas as pd
import re
import networkx as nx
import plotly.express as px
import requests

# setup plotting for quarto
alt.renderers.enable('default')
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

# supress warnings
import warnings
warnings.filterwarnings('ignore')

# 1. Motivation 

The Beatles are my favorite band, and I know that their music underwent rapid and profound change during the 1960s.  The albumns *Rubber Soul* (1965) and *Revolver* (1966) held pivotal places in this evolution, as Walter Everett explained in his excellent 1999 book, *The Beatles as Musicians:  Revolver to The Anthology*.  So I was interested to see whether this change was in any way evident in the kind of "music feature" data derived from the recordings themselves, in this case by Spotify.  

Can we see any trends across the Beatles output based on the sound itself?  Can we find any albumns that stand out from others?  And how would the evidence provided by Spotify's machine analysis tools align with my own impressions of the music, or Everett's?

# 2. Implementation

Prof. Freedman has conveniently assembled the Spotify data on all of the Beatles albumns as a single CSV file.  So this would be a simple starting place.  But since I am still a bit uncertain about the what Spotify's audio features are like, I thought the best first step would be to look at the columns in that data set and see which of them might be good candidates to tell me a bit more about the "range" of ratings for each.  If I am looking for change, then I need features (or combinations of features) that have the widest possibility for variation.

I looked at the columns as a list:  `beatles_spotify.columns`, which yielded (among the music feature categories) `['danceability', 'energy', 'speechiness','acousticness', 'liveness', 'valence']`.  Based on my exploration of the Spotify documentation and Nick Seaver's *Computing Taste Algorithms and the Makers of Music Recommendation*  I thought that 'danceability', 'energy', and 'acousticness' seemed the the categores most likely to reveal the trends I was interested in, particularly since (as I learned from Everett) the Beatles' recordings moved from a concert and dance music context to a contemplative, narrative and listening one. I guessed that I might see a decline in danceability over time and also a decline in acousticness (as more studio techniques came to the fore).

As explained in the code cells below, I broke this down into several exploratory steps:

2A.  I looked at the range of data for each audio feature, in this case by cutting the data for all tracks into 'bins' for each column using the Pandas 'cut' method.  There were some clear trends here, but since this was at the *track* level it was hard for me to say much about individual albumns over time.

2B.  Then I came up with ways of aggregating the data for each album using Pandas `groupby` operations.  These are a bit tricky to understand, but using this I was easily able to say something about each.  The key think with `groupby` is to remember that you need to pass in a *list* of the columns, and then specify the function to apply to each of them (in this case:  `mean`.  Resetting the index gets the year and album names back in the dataframe and makes the whole easier to parse in subsequent steps.  The complete code is:

```python
grouped_data = beatles_spotify.groupby(['year', "album"])[['danceability', 'energy', 'acousticness']].mean()
grouped_data = grouped_data.reset_index()
```

2C.  The resulting dataframe of results was hard for me to interpret, so I hit upon the idea of showing a kind of 'triple bar chart' for each album, with a consistent color for each feature. This was useful, since it allowed me to see what was distinctive about certain records as a whole.  *Rubber Soul* stood out on account of the *divergence* between the audio features (it hasd a much *lower* score for 'acousticness' in relation to the other features than any other album.

2D.  This in turn prompted me to make a scatterplot for this album on its own and then to test those results against my own impressions and those of my authors.

The commented code for each of these appears in the relevant cells below.







#### Getting the Data from the CSV


In [ ]:
# the address of the shared CSV provided by Prof Freedman
beatles_spotify_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRCv45ldJmq0isl2bvWok7AbD5C6JWA0Xf1tBqow5ngX7_ox8c2d846PnH9iLp_SikzgYmvdPHe9k7G/pub?output=csv'
# import that to a Pandas dataframe and name it
beatles_spotify = pd.read_csv(beatles_spotify_csv)

#### Checking the column names


In [ ]:
# find the columns in the dataframe
beatles_spotify.columns

#### Clean up NaNs for now


In [ ]:
# some precautionary cleanup in case of NAs
beatles_spotify.fillna('').head()

#### 2a: An informal way to review the range of values in each feature, over all albums


In [ ]:
# and a very informal way to look at my columns to get a sense of the range of values across the albumns
# below I use the pandas "cut" method to split the data into 'bins' of similar values

# my preferred cols as a lis
column_list = ['danceability', 'energy', 'acousticness']
# a for loop
for col in column_list:
    # show the 'binned' values for each feature column
    print(pd.cut(beatles_spotify[col], bins=4).value_counts())

#### 2b: Groupby operation using year/album, then the chosen audio features to find the mean values for each


In [ ]:
grouped_data = beatles_spotify.groupby(['year', "album"])[['danceability', 'energy', 'acousticness']].mean()
grouped_data = grouped_data.reset_index()
grouped_data.style.set_caption("Table 1: Selected Mean Audio Feature Values for Beatles Albums").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'black'),
        ('font-size', '18px')
    ]
}])

#### 2c: A Plotly barchart to show the comparative values for each in a graphical way


In [ ]:
fig = px.bar(grouped_data,
             x='album',
             y=['danceability', 'energy', 'acousticness'],
             labels={'danceability': 'Danceability', 'energy': 'Energy', 'acousticness': 'Acousticness'},
             title='Figure 1: Compararive Beatles Album Scores for Selected Audio Features',
             color_discrete_sequence=px.colors.qualitative.Pastel)

# Update layout for better visualization
fig.update_layout(barmode='group', width=800, height=600)

# Show the figure
fig.show()

#### 2d:  Exploring Rubber Soul Features


In [ ]:
# filtering full dataset for just this album
rubber_soul = beatles_spotify[beatles_spotify['album'] == 'Rubber Soul']
# create table and caption
rubber_soul.style.set_caption("Table 2: Rubber Soul Audio Features").set_table_styles([{
    'selector': 'caption',
    'props': [
        ('color', 'black'),
        ('font-size', '18px')
    ]
}])

#### A Plotly Scatterplot for Rubber Soul


In [ ]:
fig = px.scatter(rubber_soul, 
                 x='song', 
                 y=['danceability', 'acousticness', 'energy'], 
                 color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title_text="Figure 2: Scatter Plot of Rubber Soul Songs")
fig.show()

# 3. Interpretation

I was both surprised by and curious to explore the results.  Based on the audio feature data I selected, *Rubber Soul* is clearly an outlier among the other album on account of the very wide gulf between the mean 'danceability and energy' on one hand and the 'acousticness' on the other.  Of course we cannot be completely how the Spotify assigns a single value to a track (much less the 'mean' scores that I derived from those individual pieces). But the *differential* is nevertheless striking.  What might it mean?

A low score in 'acousticness' might be another way of saying that these pieces are *more* electric, or at least more characterized by enriched timbres of the sort one could produce in the studio.  And indeed this is one of the important trends that marks the Beatles' development as musicians during this critical phase of their careers as a *studio* band.

So with this insight in mind, I decided to look more closely at the data for individual songs in *Rubber Soul* and test these against my own impressions of the music (and against what Everett said).  What is fascinating to me is that many of the songs that are characterized by Spotify as 'low' in acousticness still involve a good deal of very conventional instruments, voices, and even volume.  They are nevertheless filled with blended voices, and very diverse timbres (sitar, pseudo harpsichord, echo effects, and so on).  This is very interesting, and as Everett notes, reflects the increasing concern of the Beatles to use the studio itself as a kind of instrument, quite apart from the actual instruments and voices involved in the songs!


# 4.  Bibliography

Everett, Walter. 1999.  *The Beatles as Musicians: Revolver to The Anthology*.  Oxford and New York:  Oxford University Press.

Seaver, Nick. 2022.  *Computing Taste:  Algorithms and the Makers of Music Recommendation*.  Chicago:  University of Chicago Press.
